# Usage 
This notebook visualizes how CXIL can be utilized. 

In [ ]:
# Load the Data

In [ ]:
# Initialize a Model 

In [ ]:
# Choose a Learner

In [ ]:
# Choose the Interaction Typr

In [ ]:
# Run 

In [ ]:
# Evaluate